In [1]:
from utils import PySubmissionFormatter, RsSubmissionFormatter
import pandas as pd
import re

In [2]:
data = pd.read_csv('data/leetcode_hard_with_snippets_clean.csv')

### Python Dataset

In [11]:
function_name_regex = r"(?<=def\s)\w+"
lines = []

for ind, row in data.iterrows():
    task_id = row['title_slug']
    comment = '\n'.join([f'# {s}' for s in row['description'].strip().split('\n')])
    prompt = PySubmissionFormatter().to_humaneval(comment + '\n' + row['python3_code_snippet']).strip('\n') + '\n'
    entry_point = re.search(function_name_regex, row['python3_code_snippet']).group(0)
    
    line = {
        'task_id': entry_point,
        'prompt': prompt,
        'entry_point': task_id,
        'cannonical_solution': '',
        'test': ''
    }
    
    lines.append(line)


In [12]:
import json
import os

def save_dict_to_jsonl(dict_data, file_path):
    with open(file_path, 'a') as file:
        json_line = json.dumps(dict_data)
        file.write(json_line + os.linesep)

def save_list_of_dicts_to_jsonl(list_of_dicts, file_path):
    for dict_data in list_of_dicts:
        save_dict_to_jsonl(dict_data, file_path)

In [13]:
save_list_of_dicts_to_jsonl(lines, 'data/leetcode-hard-py.jsonl')

In [16]:
save_dict_to_jsonl(lines, )

In [ ]:
# task_id
# prompt
# entry_point
# canonical_solution
#test

In [36]:
import re

class PySubmissionFormatter(SubmissionFormatter):
    def to_leetcode(self, humaneval_snippet: str):
        comment_pattern = re.compile(r"((?:#.*\n)*)")
        comment_match = comment_pattern.match(humaneval_snippet)
        comments = comment_match.group(1) if comment_match else ""
        
        # Remove comments from the snippet
        humaneval_snippet = comment_pattern.sub("", humaneval_snippet).strip()
        humaneval_snippet_indented = humaneval_snippet.replace('\n', '\n    ')
        
        return f"""\
{comments}class Solution:
    {humaneval_snippet_indented}
"""

    def to_humaneval(self, leetcode_snippet: str):
        comment_pattern = re.compile(r"((?:#.*\n)*)")
        comment_match = comment_pattern.match(leetcode_snippet)
        comments = comment_match.group(1) if comment_match else ""
        
        # Remove comments from the snippet
        leetcode_snippet = comment_pattern.sub("", leetcode_snippet).strip()
        
        pattern = re.compile(r"class Solution:\s+([\s\S]+)")
        match = pattern.search(leetcode_snippet)
        if match:
            return f"{comments}{match.group(1).replace('    ', '', 1)}"
        return leetcode_snippet.strip()


In [37]:
humaneval_snippet_1 = "# This is a comment\n# Another comment\ndef function_name():\n    pass"

formatted_leetcode_1 = PySubmissionFormatter().to_leetcode(humaneval_snippet_1)
print(formatted_leetcode_1)



# This is a comment
# Another comment
class Solution:
    def function_name():
        pass

